<a href="https://colab.research.google.com/github/heesukjang/W266_NLP_With_DeepLearning/blob/main/Final_Project/EDA_with_Baseline_SimpleBERT_Automated_Essay_Evaluation_v2_NatalieOH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Baseline: A Simple Pre-Trained BERT for the Text Classification - Automated 

To set up a BERT model for a classification task using essays as an input, you can follow these general steps:

1. Data preprocessing: Preprocess the essays by tokenizing them into individual words and applying any necessary cleaning or normalization techniques.

2. Data splitting: Split the preprocessed essays into training, validation, and test sets.

3. Model architecture: Choose a pre-trained BERT model (e.g., BERT base, BERT large) and adapt it for your classification task by adding a classification layer on top of the BERT output.

4. Fine-tuning: Fine-tune the pre-trained BERT model on the training set by minimizing a suitable loss function. During fine-tuning, the weights of the BERT model will be updated to better fit the specific classification task.

5. Evaluation: Evaluate the fine-tuned model on the validation set to tune any hyperparameters (e.g., learning rate, number of epochs). After finalizing the hyperparameters, evaluate the model on the test set to obtain the final performance metrics.

6. Inference: Use the trained BERT model to predict the classification label for new essays by passing them through the preprocessed and fine-tuned model.

There are many libraries and frameworks available to help you set up a BERT model for classification tasks, such as Hugging Face's Transformers library in Python. You can find many tutorials and code examples online to help you get started.

## A sample Python code to train a BERT model for text classification using the PyTorch library:

In [23]:
# import torch
# import torch.nn as nn
# from transformers import BertModel, BertTokenizer
# from torch.utils.data import DataLoader, Dataset

# class MyDataset(Dataset):
#     def __init__(self, data, tokenizer, max_len):
#         self.tokenizer = tokenizer
#         self.data = data
#         self.max_len = max_len
    
#     def __len__(self):
#         return len(self.data)
    
#     def __getitem__(self, index):
#         text = self.data[index]['text']
#         label = self.data[index]['label']
#         encoding = self.tokenizer.encode_plus(
#             text,
#             add_special_tokens=True,
#             max_length=self.max_len,
#             padding='max_length',
#             truncation=True,
#             return_attention_mask=True,
#             return_tensors='pt',
#         )
#         return {
#             'text': text,
#             'input_ids': encoding['input_ids'].flatten(),
#             'attention_mask': encoding['attention_mask'].flatten(),
#             'label': torch.tensor(label, dtype=torch.long)
#         }

# class BERTClassifier(nn.Module):
#     def __init__(self, num_classes):
#         super(BERTClassifier, self).__init__()
#         self.bert = BertModel.from_pretrained('bert-base-uncased')
#         self.dropout = nn.Dropout(p=0.1)
#         self.classifier = nn.Linear(self.bert.config.hidden_size, num_classes)
    
#     def forward(self, input_ids, attention_mask):
#         output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
#         output = self.dropout(output.pooler_output)
#         output = self.classifier(output)
#         return output

# def train(model, train_loader, criterion, optimizer, device):
#     model.train()
#     train_loss = 0.0
#     correct = 0
#     total = 0
#     for batch in train_loader:
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         label = batch['label'].to(device)
#         optimizer.zero_grad()
#         output = model(input_ids, attention_mask)
#         loss = criterion(output, label)
#         train_loss += loss.item() * input_ids.size(0)
#         loss.backward()
#         optimizer.step()
#         pred = torch.argmax(output, dim=1)
#         correct += (pred == label).sum().item()
#         total += input_ids.size(0)
#     train_acc = 100 * correct / total
#     train_loss = train_loss / total
#     return train_loss, train_acc

# def validate(model, val_loader, criterion, device):
#     model.eval()
#     val_loss = 0.0
#     correct = 0
#     total = 0
#     with torch.no_grad():
#         for batch in val_loader:
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             label = batch['label'].to(device)
#             output = model(input_ids, attention_mask)
#             loss = criterion(output, label)
#             val_loss += loss.item() * input_ids.size(0)
#             pred = torch.argmax(output, dim=1)
#             correct += (pred == label).sum().item()
#             total += input_ids.size(0)
#     val_acc = 100 * correct / total
#     val_loss = val_loss / total
#     return val_loss, val_acc

# def train_model(train_data, val_data, batch_size, max_len, num_classes, num_epochs, learning_rate, device):
#     tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#     train_dataset = MyDataset(train_data, tokenizer, max


# Regenerate response


In [24]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
!pip install wordcloud

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
import transformers
print(f'transformers version: {transformers.__version__}')
from transformers import logging as hf_logging
hf_logging.set_verbosity_error()

transformers version: 4.26.1


In [27]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud
from wordcloud import STOPWORDS
from wordcloud import ImageColorGenerator
import sys

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils.layer_utils import count_params

from tensorflow.keras.layers import RandomFlip
from tensorflow.keras.layers import RandomZoom
from tensorflow.keras.layers import RandomRotation
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling1D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import Multiply
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import PReLU
from keras.layers.core import Activation
from keras.layers.convolutional import SeparableConv1D
from keras.layers.convolutional import SeparableConv2D

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop

from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.densenet import *
from keras.applications.resnet import ResNet152
from keras.applications.nasnet import NASNetMobile
from keras.applications.nasnet import NASNetLarge
from keras.applications.efficientnet import EfficientNetB7
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.nasnet import preprocess_input
from keras.applications import MobileNetV2
from keras.applications.xception import Xception
from keras.applications.inception_v3 import *

from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import save_img

from tensorflow.python.ops.numpy_ops import np_config

from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import LearningRateScheduler

from tensorflow.keras.losses import mae
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.losses import binary_crossentropy

from keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.regularizers import l1
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD
from keras.models import load_model

import warnings
warnings.filterwarnings("ignore")

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [28]:
# def set_seed(seed = 99):
#     np.random.seed(seed)
#     tf.random.set_seed(seed)
#     os.environ['PYTHONHASHSEED'] = str(seed)
#     os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Kaggle"

# set_seed(20230214)

In [29]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_columns', None)

## Build a classifier for each rubric metric i.e. cohesion, vocab, etc
## Evaluatio metrics (possibly use ChatGPT to produce reference essays):
- BLUE score
- ROUGE score

In [37]:
train_path = '/content/gdrive/MyDrive/Kaggle/train.csv'
test_path = '/content/gdrive/MyDrive/Kaggle/test.csv'

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

Generally, the training and validation data set is split into an 80:20 ratio. Thus, 20% of the data is set aside for validation purposes. The ratio changes based on the size of the data.

In [38]:
train_df.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


In [39]:
test_df.head()

,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,Do you think students would benefit from being...
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde..."


In [42]:
# add new columns for aggregation: 'average_score', 'total_score', and 'total_score_in_perc'
print(train_df.columns)
cols = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']

# row-wise aggregation
total_points = 30
train_df['avg_score'] = train_df[cols].mean(axis=1).round(1)
train_df['total_score'] = train_df[cols].sum(axis=1)
train_df['final_score_in_perc'] = (train_df[cols].sum(axis=1)/total_points*100).round(1)

final_grades = []
for score in train_df['final_score_in_perc']:
  if score < 60:
    final_grades.append('F')
  elif score < 70:
    final_grades.append('D')
  elif score < 80:
    final_grades.append('C')
  elif score < 90:
    final_grades.append('B')
  elif score < 95:
    final_grades.append('A')
  else:
    final_grades.append('A+')    
# print(final_grades)
train_df['final_grade_letter'] = final_grades
# cat_cohesion = pd.cut(train_df['cohesion'],[1,2,3,4,5])
# train_df['cat_cohesion'] = cat_cohesion
# train_df['num_label_cohesion'] = train_df['cat_cohesion'].cat.codes
# train_df['str_label_cohesion'] = train_df.apply(lambda x: x['num_label_cohesion'])

train_df[['full_text','cohesion']].head()

Index(['text_id', 'full_text', 'cohesion', 'syntax', 'vocabulary',
       'phraseology', 'grammar', 'conventions', 'avg_score', 'total_score',
       'final_score_in_perc', 'final_grade_letter'],
      dtype='object')


,full_text,cohesion
0,I think that students would benefit from learn...,3.5
1,When a problem is a change you have to let it ...,2.5
2,"Dear, Principal\n\nIf u change the school poli...",3.0
3,The best time in life is when you become yours...,4.5
4,Small act of kindness can impact in other peop...,2.5


In [ ]:
print('Unique Values in Each Metric:\n==================================================')
for col in cols:
  print(f'{col}: {train_df[col].unique()}')

## Create 5 point likert scale for each metric


In [ ]:
train_df.shape, test_df.shape

In [ ]:
train_df.info()

In [ ]:
train_df[train_df.final_score_in_perc >= 95]

In [ ]:
train_df.describe()

In [ ]:
label_cols = train_df.columns[2:]
label_cols

In [ ]:
train_df[label_cols].drop_duplicates()

In [ ]:
train_df[label_cols].value_counts()

In [ ]:
# label_cols = label_cols[:-1]
label_cols

In [ ]:
train_df[train_df.total_score == 30][['full_text', 'avg_score', 'total_score', 'final_score_in_perc']]

In [ ]:
fig, ax = plt.subplots(1, len(label_cols), figsize=(20,4))

for idx, label in enumerate(label_cols):
    sns.histplot(x = label, 
                 data = train_df,
                 ax = ax[idx]
                )
    ax[idx].set_title(label)
    

In [ ]:
for label in label_cols[:-4]:
    train_df[label + '_above_or_below_avg_flag'] = np.where(train_df[label] > np.mean(train_df[label]), 1, 0)

In [ ]:
fig, ax = plt.subplots(1, len(label_cols[:-4]), figsize=(50,10))
for idx, label in enumerate(label_cols[:-4]):
    sns.countplot(x = train_df[label + '_above_or_below_avg_flag'], ax = ax[idx])
    ax[idx].set_title(label)

In [ ]:
corr = train_df[label_cols].corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype = bool))

sns.set(rc = {"figure.figsize": (10, 8)})

sns.heatmap(corr, 
            annot = True, 
            cmap = "coolwarm", 
            mask = mask,
            fmt  = ".2f")
plt.show()

In [ ]:
# corr bt word count and score in each category

In [ ]:
first_essay = train_df['full_text'][0]
print(f'In the First Essay\n=====================================\nnum_words (tokens) = {len(first_essay.split())}\nnum_characters_in_first_essay = {len(first_essay)}\n=====================================')
first_essay

In [ ]:
fig = plt.figure(figsize=(15, 5))
train_df['word_count'] = train_df['full_text'].apply(lambda x: len(x.split()))
sns.histplot(data = train_df, x = "word_count")
plt.title("full_text word count histogram plot")
plt.show()

In [ ]:
# mean > median
plt.figure(figsize = (15,5))
sns.distplot(train_df['word_count'])
plt.axvline(x = np.mean(train_df['word_count']), color = 'red')
plt.title('full_text length distribution plot')

print(f'Average word count = {int(train_df.word_count.mean())}')

In [ ]:
# options to explore:
# - truncate (anything longer than 512 max length)
# - split each essay to multiple essay and average (median or mode) them (to see a whole essay)

In [ ]:
fig = plt.figure(figsize=(15, 5))
train_df['full_text_len'] = train_df['full_text'].apply(lambda x: len(x))
sns.histplot(data = train_df, x = "full_text_len")
plt.title("full_text length histogram plot")
print(f'Average length of letter sequences in essays = {int(train_df.full_text_len.mean())}')
plt.show()

In [ ]:
plt.figure(figsize = (15,5))
sns.distplot(train_df['full_text_len'])
plt.axvline(x = np.mean(train_df['full_text_len']), color = 'red')
plt.title('full_text length distribution plot')

In [ ]:
train_df.describe()

In [ ]:
text = train_df[(train_df.cohesion == 5) & 
                (train_df.syntax == 5) & 
                (train_df.vocabulary == 5) & 
                (train_df.phraseology == 5) & 
                (train_df.grammar == 5) & 
                (train_df.conventions == 5)]['full_text'].values[0]
word_cloud = WordCloud(max_font_size = 50, max_words = 100, background_color = "white").generate(text)
plt.imshow(word_cloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
#make it easier to use a variety of BERT subword models
model_checkpoint = 'bert-base-cased'   # case sensitive (care about upper and lower case)

In [ ]:
from transformers import BertTokenizer, TFBertModel

bert_tokenizer = BertTokenizer.from_pretrained(model_checkpoint)
bert_model = TFBertModel.from_pretrained(model_checkpoint)

In [ ]:
train_texts = train_df['full_text']
train_texts

In [ ]:
# print(f'len(train_texts)={len(train_texts)} | type(train_texts)={type(train_texts)}')
train_texts = train_texts.values.tolist()
print(f'len(train_texts)={len(train_texts)} | type(train_texts)={type(train_texts)}')

In [ ]:
train_encodings = bert_tokenizer(train_texts, truncation=True, padding=True, max_length=200, return_tensors='tf')

In [ ]:
print(f'len(train_encodings.input_ids[:1] = {len(train_encodings.input_ids[:1])}')
# Average length of sequences in essays = 2334



- 6 models for each score for 6 categories: ordinal logistic regression.
- seq to seq for multi-output model

**Natalie's comment:**<br>
1) build a classifier for each metric OR<br>
2) build a multi-label multi-class model where you set up 6 output classification layer with one input tokenized by BERT tokenizer (maybe use CLS token layer for our input layer)




In [ ]:
def create_simple_bert(checkpoint = model_checkpoint,
                       num_classes=9,   # [1, 1.5, 2, 2.5....4.5, 5]: 9 classes
                       hidden_size=201,
                       dropout=0.3,
                       learning_rate=0.00005,
                       max_length=512):
    """
    Build a simple classification model with BERT. Use the Pooler Output for classification purposes.
    """
    bert_model = TFBertModel.from_pretrained(checkpoint)                                              
    bert_model.trainable = False     

    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}

    bert_out = bert_model(bert_inputs)   # full features

    # ======== revise to utilize logistic regression =================
    pooler_output = bert_out[1]    # one vector for each  
    # cls token layer

    hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(pooler_output)   # may not be needed
    hidden = tf.keras.layers.Dropout(dropout)(hidden)  

    classification = tf.keras.layers.Dense(num_classes, activation='softmax',name='classification_layer')(hidden)    
    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])

    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                 loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),          # A 'sigmoid' activation layer turns the output "logit" into a value, 0-1 
                                 metrics='accuracy')                                        # work for both string and integer based labels - if your data is imbalanced, you would want F1 score or BLUE for the text classification
                                #  metrics=tf.keras.metrics.SparseCategoricalAccuracy())    # only for the integer based labels
    ### END YOUR CODE
    return classification_model